In [0]:
# Required each time the cluster is restarted which should be only on the first notebook as they run in order

tiers = ["bronze", "silver", "gold"]
adls_paths = {tier: f"abfss://{tier}@databrickstorage17.dfs.core.windows.net/" for tier in tiers}

# Accessing paths
bronze_adls = adls_paths["bronze"]
silver_adls = adls_paths["silver"]
gold_adls = adls_paths["gold"] 

dbutils.fs.ls(bronze_adls)
dbutils.fs.ls(silver_adls)
dbutils.fs.ls(gold_adls)

[]

In [0]:
import requests
import json
from datetime import date, timedelta

In [0]:
start_date = date.today() - timedelta(1)
end_date = date.today()

In [0]:
# Construct the API URL with start and end dates provided by Data Factory, formatted for geojson output.
url = f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={start_date}&endtime={end_date}"

try:
    # Make the GET request to fetch data
    response = requests.get(url)

    # Check if the request was successful
    response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
    data = response.json().get('features', [])

    if not data:
        print("No data returned for the specified date range.")
    else:
        # Specify the ADLS path
        file_path = f"{bronze_adls}/{start_date}_earthquake_data.json"

        # Save the JSON data
        json_data = json.dumps(data, indent=4)
        dbutils.fs.put(file_path, json_data, overwrite=True)
        print(f"Data successfully saved to {file_path}")
except requests.exceptions.RequestException as e:
    print(f"Error fetching data from API: {e}")

Wrote 387462 bytes.
Data successfully saved to abfss://bronze@databrickstorage17.dfs.core.windows.net//2025-03-07_earthquake_data.json


In [0]:
data

[{'type': 'Feature',
  'properties': {'mag': 2.3,
   'place': '16 km NE of Stanley, Idaho',
   'time': 1741389237870,
   'updated': 1741392633820,
   'tz': None,
   'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/mb90073713',
   'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=mb90073713&format=geojson',
   'felt': None,
   'cdi': None,
   'mmi': None,
   'alert': None,
   'status': 'reviewed',
   'tsunami': 0,
   'sig': 81,
   'net': 'mb',
   'code': '90073713',
   'ids': ',mb90073713,',
   'sources': ',mb,',
   'types': ',origin,phase-data,',
   'nst': 24,
   'dmin': 0.2364,
   'rms': 0.21,
   'gap': 46,
   'magType': 'ml',
   'type': 'earthquake',
   'title': 'M 2.3 - 16 km NE of Stanley, Idaho'},
  'geometry': {'type': 'Point',
   'coordinates': [-114.822333333333, 44.342, 7.08]},
  'id': 'mb90073713'},
 {'type': 'Feature',
  'properties': {'mag': 3.67,
   'place': '99 km N of Cruz Bay, U.S. Virgin Islands',
   'time': 1741389088390,
   'updated': 17413

In [0]:
data[0]

{'type': 'Feature',
 'properties': {'mag': 2.3,
  'place': '16 km NE of Stanley, Idaho',
  'time': 1741389237870,
  'updated': 1741392633820,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/mb90073713',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=mb90073713&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'reviewed',
  'tsunami': 0,
  'sig': 81,
  'net': 'mb',
  'code': '90073713',
  'ids': ',mb90073713,',
  'sources': ',mb,',
  'types': ',origin,phase-data,',
  'nst': 24,
  'dmin': 0.2364,
  'rms': 0.21,
  'gap': 46,
  'magType': 'ml',
  'type': 'earthquake',
  'title': 'M 2.3 - 16 km NE of Stanley, Idaho'},
 'geometry': {'type': 'Point',
  'coordinates': [-114.822333333333, 44.342, 7.08]},
 'id': 'mb90073713'}

In [0]:
# Define your variables
output_data = {
    "start_date": start_date.isoformat(),
    "end_date": end_date.isoformat(),
    "bronze_adls": bronze_adls,
    "silver_adls": silver_adls,
    "gold_adls": gold_adls
}

# Return the dictionary directly
dbutils.jobs.taskValues.set(key = "bronze_output", value = output_data)